<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/person_cigarette_model_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run a model for sigarette and person detection

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from ultralytics import YOLO
import torch
from pathlib import Path
import cv2
import numpy as np
from tqdm.auto import tqdm
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class InferenceVisualizer:
    def __init__(self, model_path, conf_threshold=0.25):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_path)
        self.model.to(self.device)
        self.conf_threshold = conf_threshold

        self.colors = {
            0: (0, 255, 0),    # Verde
            1: (255, 0, 0),    # Blu
            2: (0, 0, 255),    # Rosso
        }

    def process_image(self, image_path, output_path):
        """Processa una singola immagine e salva il risultato"""
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Impossibile leggere l'immagine: {image_path}")
            return

        results = self.model.predict(image, conf=self.conf_threshold)[0]

        annotated_image = image.copy()

        boxes = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        confidences = results.boxes.conf.cpu().numpy()

        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box.astype(int)
            class_id = int(cls)
            color = self.colors.get(class_id, (0, 255, 0))

            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)

            label = f"Class {class_id}: {conf:.2f}"

            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_x = x1
            text_y = y1 - 5 if y1 - 5 > text_size[1] else y1 + text_size[1]

            cv2.rectangle(annotated_image,
                        (text_x, text_y - text_size[1] - 4),
                        (text_x + text_size[0], text_y),
                        color, -1)

            cv2.putText(annotated_image, label,
                       (text_x, text_y - 2),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                       (255, 255, 255), 2)

        cv2.imwrite(str(output_path), annotated_image)

    def process_directory(self, input_dir, output_dir):
        """Processa tutte le immagini in una directory"""
        input_path = Path(input_dir)
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        image_files = list(input_path.glob('*.jpg')) + \
                     list(input_path.glob('*.jpeg')) + \
                     list(input_path.glob('*.png'))

        print(f"Trovate {len(image_files)} immagini")

        for img_path in tqdm(image_files, desc="Processing images"):
            output_file = output_path / f"detected_{img_path.name}"
            self.process_image(img_path, output_file)

In [5]:
def main():
    model_path = '/content/drive/MyDrive/pth_person_detect/run_20250211_142722/train/weights/best.pt'
    input_dir = '/content/drive/MyDrive/Photo person/test/images'
    output_dir = '/content/drive/MyDrive/test_trained_person'

    visualizer = InferenceVisualizer(model_path, conf_threshold=0.6)

    visualizer.process_directory(input_dir, output_dir)

    print("Elaborazione completata!")
    print(f"Le immagini elaborate sono state salvate in: {output_dir}")

if __name__ == "__main__":
    main()

Trovate 73 immagini


Processing images:   0%|          | 0/73 [00:00<?, ?it/s]


0: 576x1024 2 cigarettes, 1 nonSmoker, 2 smokers, 706.1ms
Speed: 33.5ms preprocess, 706.1ms inference, 5.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 cigarettes, 1 nonSmoker, 3 smokers, 548.7ms
Speed: 10.2ms preprocess, 548.7ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 cigarettes, 1 nonSmoker, 3 smokers, 517.1ms
Speed: 11.1ms preprocess, 517.1ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 cigarette, 1 nonSmoker, 370.4ms
Speed: 7.1ms preprocess, 370.4ms inference, 0.5ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 nonSmoker, 3 smokers, 382.8ms
Speed: 8.3ms preprocess, 382.8ms inference, 0.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 cigarette, 1 nonSmoker, 2 smokers, 351.6ms
Speed: 6.7ms preprocess, 351.6ms inference, 2.8ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 6 nonSmokers, 1 smoker, 381.3ms
Speed: 13.7ms preprocess, 381.3